In [35]:
# New notebook because Folium crashed the studnet.ipynb
import pandas as pd
import numpy as np
from scipy import stats
df = pd.read_csv('data/kc_house_data.csv')

In [36]:
# Functions to help convert our object datatypes into int64 / float
# Convert 'waterfront' column to 0's and 1's to represent 'NO' and 'YES'
def map_waterfront_bool(arr):
    if arr == 'YES':
        return 1
    elif arr == 'NO':
        return 0

# Convert 'condition' column into an int64 format from object
def condition_conv(vals):
    if vals == 'Very Good':
        return 5
    elif vals == 'Good':
        return 4
    elif vals == 'Average':
        return 3
    elif vals == 'Fair':
        return 2
    elif vals == 'Poor':
        return 1

# Convert the '?' from "sqft_basement" column to 0 (best to assume they do not have a basement)
def sqft_question(sqft):
    if sqft == '?':
        return 0
    else:
        return sqft

# Convert all 'NONE' views to 0's in order to represent it in a boolean form
def view_bool(views):
    if views == 'NONE':
        return 0
    else:
        return 1

def basement_conv(basement):
    if basement > 0:
        return 1
    else:
        return 0
    
def renovate(yr_reno):
    if yr_reno > 2001:
        return 1
    else:
        return 0

def bool_to_num(x):
    if x:
        return 1
    else:
        return 0

# Convert 'date' column into int64 with year | Unnecessary as we dont really care about the posted date
# Columns for renovation year and built year are much more relevant.
df['date'] = df['date'].str[5:]
df['date'] = df['date'].str.replace('/','').astype(np.int64)

In [37]:
# Call all functions to turn data into a manipulatable format
# First fill missing values in 'waterfront' column to 'NO'
df['waterfront'].fillna('NO', inplace = True)
df['waterfront'] = df['waterfront'].map(map_waterfront_bool)
df['condition'] = df['condition'].map(condition_conv)
df['sqft_basement'] = df['sqft_basement'].map(sqft_question).astype(float)
df['grade'] = df['grade'].str[:2].astype(np.int64)
df['view'] = df['view'].map(view_bool)
df['sqft_basement'] = df['sqft_basement'].map(basement_conv)
# Rename column to more accurately label our new datatype
df.rename({'sqft_basement': 'has_basement'}, inplace = True, axis = 1)
df['yr_renovated'] = df['yr_renovated'].map(renovate)

# Defining a limitation so Folium doesn't crash this notebook
df_below_par = df[df['condition'] < 3]

In [38]:
df['living_area'] = df['sqft_living'] / df['sqft_living15']

In [39]:
df_prospects = df[(np.abs(stats.zscore(df['price'])) < 3)].copy(deep=True)
df_prospects = df[(np.abs(stats.zscore(df['bedrooms'])) < 3)].copy(deep=True)
df_prospects = df[(np.abs(stats.zscore(df['floors'])) < 3)].copy(deep=True)
df_prospects = df[(np.abs(stats.zscore(df['sqft_living'])) < 3)].copy(deep=True)
df_prospects = df[(np.abs(stats.zscore(df['bathrooms'])) < 3)].copy(deep=True)
df_prospects = df[(np.abs(stats.zscore(df['sqft_lot'])) < 3)].copy(deep=True)

df_prospects['Seattle'] = (df_prospects['lat'] < 47.777799)   & (df_prospects['lat'] > 47.503347) &\
                          (df_prospects['long'] > -122.419374) & (df_prospects['long'] < -122.255819)
df_prospects['Seattle'] = df_prospects['Seattle'].map(bool_to_num)

df_prospects.to_csv('data/cleaned_data.csv')
df_prospects.shape

(21251, 23)